# CNN


CNN 通常用于计算机视觉任务。CNN 通过在输入数据上滑动卷积核来提取特征。卷积核是一个小的矩阵，它在输入数据上滑动，计算输入数据和卷积核之间的点积。卷积核的大小通常是 3x3 或 5x5。卷积核的大小和数量是超参数，需要根据具体任务进行调整。CNN 的变体有： ResNets、DenseNets 和 Octave 卷积。

## Neural Networks and Representation Learning

在之前的章节中，展示了两个例子，第一个是房屋预测，第二个是 MNIST 预测。在这两个例子中，带隐藏层的神经网络效果均要好于不带隐藏层的神经网络。原因是隐藏层提取了输入数据和输出数据之间的非线性的特征。

然而，一个更普遍的原因是，在机器学习中，我们经常需要原始特征的线性组合，以便有效地预测我们的目标。例如在 MNIST 中，单张图片有 784 个像素，不同像素的组合可以表示不同的数字，所有这些组合都会对图像是某一特定数字的概率产生积极或消极的影响。神经网络可以通过训练过程自动发现重要的原始特征组合。这一过程首先是通过与随机权重矩阵相乘来创建原始特征的初始随机组合；通过训练，神经网络学会提炼出有用的组合，并摒弃那些没用的组合。这种学习哪些特征组合是重要的过程被称为 `表征学习`，也是神经网络在不同领域取得成功的主要原因。

![CNN](./images/05_cnn1.png)


### A Different Architecture for Image Data


同样也是创建特征的组合，但是是在更高层次并且是更高的数量级上。每个特征的组合只是输入图像中一小块矩形区域的像素组合。

![CNN](./images/05_cnn2.png)

让神经网络学习所有特征的组合，也就是所有像素的组合是困难且低效的，且忽略了前面的一个见解（Insight）：图像中大部分重要的特征组合都出现在小矩形图像中。从这些小矩形中抽取特征并组合的操作是卷积操作。卷积操作是一种特殊的矩阵乘法，它在输入数据上滑动一个小的矩阵（卷积核），计算输入数据和卷积核之间的点积。卷积核的大小通常是 3x3 或 5x5。卷积核的大小和数量是超参数，需要根据具体任务进行调整。


### The Convolution Operation


在设定卷积核大小后，卷积核应用在图像上的不同位置。例如我们有以下输入图像的数据：
$$
I = \begin{bmatrix}
i_{1,1} & i_{1,2} & i_{1,3} & i_{1,4} & i_{1,5} \\
i_{2,1} & i_{2,2} & i_{2,3} & i_{2,4} & i_{2,5} \\
i_{3,1} & i_{3,2} & i_{3,3} & i_{3,4} & i_{3,5} \\
i_{4,1} & i_{4,2} & i_{4,3} & i_{4,4} & i_{4,5} \\
i_{5,1} & i_{5,2} & i_{5,3} & i_{5,4} & i_{5,5} \\
\end{bmatrix}
$$

假设我们有一个 3x3 的卷积核：
$$
K = \begin{bmatrix}
k_{1,1} & k_{1,2} & k_{1,3} \\
k_{2,1} & k_{2,2} & k_{2,3} \\
k_{3,1} & k_{3,2} & k_{3,3} \\
\end{bmatrix}
$$

假设我们将卷积核作用在 $I$ 中间的 (3,3) 位置，结果记为 $o_33$ （o 表示 output）
$$
o_{33} = w_{1,1} \times i_{2,2} + w_{1,2} \times i_{2,3} + w_{1,3} \times i_{2,4} + w_{2,1} \times i_{3,2} + w_{2,2} \times i_{3,3} + w_{2,3} \times i_{3,4} + w_{3,1} \times i_{4,2} + w_{3,2} \times i_{4,3} + w_{3,3} \times i_{4,4}
$$

这个值会像我们在神经网络中看到的其他计算特征一样被处理：它可能会被添加一个偏置，然后可能会被送入一个激活函数，然后它将代表一个 "神经元 "或 "学习特征"，并传递给网络的后续层

根据不同卷积核的类型，也称为"模式提取"（pattern detectors），可以提取图像中不同属性的特征。例如下面的 3x3 卷积核可以检测图像中的边缘：
$$
K = \begin{bmatrix}
0& 1 & 0 \\
1 & 4 & 1 \\
0 & 1 & 0 \\
\end{bmatrix}
$$

已知还有类似的矩阵可以检测是否存在角、是否存在垂直线或水平线，等等。这些卷积核的选择是超参数，需要根据具体任务进行调整。

现在假设我们使用同一组权重W来检测由W定义的视觉模式是否存在于输入图像中的每个位置。我们可以想象“在输入图像上滑动 W”，取 W 与图像每个位置处的像素的点积，最终得到一个与原始图像大小几乎相同的新图像 O（可能略有不同，具体取决于关于我们如何处理边缘）。图像 O 将是一种 "特征图"，显示输入图像中出现 W 所定义图案的位置。这种操作实际上就是卷积神经网络中的操作；它被称为卷积，其输出实际上被称为特征图。

在进入 `Operation` 前，我们给卷积再增加点维度。

## Multichannel Convolution Operation

回顾一下：卷积神经网络与普通神经网络的不同之处在于，它们创建的特征要多出一个数量级，而且每个特征都是输入图像中一小块区域的函数。现在我们可以说得更具体一些：从 n 个输入像素开始，刚才描述的卷积操作将创建 n 个输出特征，输入图像中的每个位置（区域）都有一个特征。

在神经网络的卷积层中实际发生的情况则更进一步：在这里，我们将创建 n 个特征的 f 个集合，每个集合都有相应的（初始随机的）权重集，这些权重集定义了一种视觉模式，输入图像中每个位置的检测结果都将被捕捉到特征图中。这 f 个特征图将通过 f 次卷积操作创建。这些特征图将被堆叠在一起，形成一个新的多通道特征图，这个特征图将被传递到网络的下一层。

![CNN](./images/05_cnn3.png)

通过特定权重集检测到的每个 "特征集 "被称为一个特征图，而在卷积层中，特征图的数量被称为卷积层的通道数--这就是为什么卷积层所涉及的操作被称为多通道卷积。此外，f 组权重 W 被称为卷积滤波器（convolutional filters）。

## Convolutional Layers


## Training Convolutional Neural Networks